# Wine Distillation - Leverage outputs of a large model to distill a smaller one

OpenAI recently released Distillation which allows to leverage the outputs of a (large) model to fine-tune another (smaller) model. This can significantly reduce the price and the latency for specific tasks as you move to a smaller model. In this exercise we'll look at a dataset, distill the output of `gpt-4o` to `gpt-4o-mini` and show how we can get significantly better results than on a generic, non-distilled, `4o-mini`.



## Overview

This notebook contains three sections: 
1. **Assessing a baseline**: Evaluating an out of the box `gpt-4o-mini` and `gpt-4o` models and understand performance
3. **Distillation**: Store the good completions and create a dataset for fine tuning your smaller model. 
4. **Extension**: If you finished the exercise and still have some time, you can try these ideas! 

## 1. Assessing a baseline for funtion calling 

When Fine Tuning a model, it's important to understand what your starting point is. For this exercise we'll be using this [Wine Reviews Dataset](https://www.kaggle.com/datasets/zynicide/wine-reviews) from Kaggle. This dataset has a large number of rows and you're free to run this cookbook on the whole data, but to speed things up, I'll narrow down the dataset to only French wine to focus on less rows and grape varieties.

We're looking at a classification problem where we'd like to guess the grape variety based on all other criterias available, including description, subregion and province that we'll include in the prompt. It gives a lot of information to the model, you're free to also remove some information that can help significantly the model such as the region in which it was produced to see if it does a good job at finding the grape.

In [6]:
%pip install kagglehub -q


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import openai
import json
import pandas as pd
import numpy as np

client = openai.OpenAI()

In [15]:
df = pd.read_csv('data/winemag-data-130k-v2.csv')
df_france = df[df['country'] == 'France']

# Let's also filter out wines that have less than 5 references with their grape variety – even though we'd like to find those
# they're outliers that we don't want to optimize for that would make our enum list be too long
# and they could also add noise for the rest of the dataset on which we'd like to guess, eventually reducing our accuracy.

varieties_less_than_five_list = df_france['variety'].value_counts()[df_france['variety'].value_counts() < 5].index.tolist()
df_france = df_france[~df_france['variety'].isin(varieties_less_than_five_list)]

df_france_subset = df_france.sample(n=500)
df_france_subset.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
35316,35316,France,This juicy wine has red berry flavors and a se...,NaN,85,12.0,Bordeaux,Bordeaux,NaN,Roger Voss,@vossroger,Château Haut Brande 2012 Bordeaux,Bordeaux-style Red Blend,Château Haut Brande
47803,47803,France,"Perfumed with wood flavors, this is a dry wine...",F de Frédignac,88,20.0,Bordeaux,Blaye Côtes de Bordeaux,NaN,Roger Voss,@vossroger,Château Frédignac 2014 F de Frédignac (Blaye ...,Bordeaux-style Red Blend,Château Frédignac
89218,89218,France,"This is a soft wine, full of ripe apple and wh...",NaN,88,19.0,Loire Valley,Quincy,NaN,Roger Voss,@vossroger,Jean-Claude Roux 2014 Quincy,Sauvignon Blanc,Jean-Claude Roux
44351,44351,France,"Intensely rich in character, touched lightly b...",NaN,92,NaN,Bordeaux,Barsac,NaN,Roger Voss,@vossroger,Château Doisy-Védrines 2007 Barsac,Bordeaux-style White Blend,Château Doisy-Védrines
63749,63749,France,"An austere wine, serious and densely dry. It h...",NaN,84,18.0,Bordeaux,Haut-Médoc,NaN,Roger Voss,@vossroger,Château Lanessan 2011 Haut-Médoc,Bordeaux-style Red Blend,Château Lanessan


In [16]:
varieties = np.array(df_france['variety'].unique()).astype('str')
varieties

array(['Gewürztraminer', 'Pinot Gris', 'Gamay',
       'Bordeaux-style White Blend', 'Champagne Blend', 'Chardonnay',
       'Petit Manseng', 'Riesling', 'White Blend', 'Pinot Blanc',
       'Alsace white blend', 'Bordeaux-style Red Blend', 'Malbec',
       'Tannat-Cabernet', 'Rhône-style Red Blend', 'Ugni Blanc-Colombard',
       'Savagnin', 'Pinot Noir', 'Rosé', 'Melon',
       'Rhône-style White Blend', 'Pinot Noir-Gamay', 'Colombard',
       'Chenin Blanc', 'Sylvaner', 'Sauvignon Blanc', 'Red Blend',
       'Chenin Blanc-Chardonnay', 'Cabernet Sauvignon', 'Cabernet Franc',
       'Syrah', 'Sparkling Blend', 'Duras', 'Provence red blend',
       'Tannat', 'Merlot', 'Malbec-Merlot', 'Chardonnay-Viognier',
       'Cabernet Franc-Cabernet Sauvignon', 'Muscat', 'Viognier',
       'Picpoul', 'Altesse', 'Provence white blend', 'Mondeuse',
       'Grenache-Syrah', 'G-S-M', 'Pinot Meunier', 'Cabernet-Syrah',
       'Vermentino', 'Marsanne', 'Colombard-Sauvignon Blanc',
       'Gros and Peti

In [17]:
########## ToDo: Test how well 4o-mini and 4o can classify these wines based on the reviews. ##########

## 2. Distillation 

It is very clear that 4o performs better than 4o-mini for this type of task. Now let's see if we can use our completions for 4o to distill the 4o-mini model. [This section](https://platform.openai.com/docs/guides/evals/generate-datasets-from-real-traffic) of our docs shows how you can start storing your completions. 

In [18]:
########## ToDo: Code to store your completions ##########

Now you need to go into your [ChatCOmpletions](https://platform.openai.com/chat-completions) front end, create your dataset and start a distillation process. 

In [19]:
########## ToDo: Code to retrieve your fine tune job based on its ft id.  ##########

In [20]:
########## ToDo: Code to check the performance of your distilled model.  ##########

## 3. Extentions 

If you've already completed the execise above, congratulations! Here are a few ideas on how to turn this into a more exciting project: 

1. Compare distillation with simple fine tuning. Anything you're noticing? 
2. Build a RAG system that can give you wine recommendations based on a customer request. 
3. Build a front end application where you can visualise this chatbot. 

## Conclusion 

Congratulations on getting this far! Now you have a better understanding of what distillation is. You can think about more usecases where distillation may be useful. Keen to see what you'll be building! 